## Import Stuff

In [ ]:
%matplotlib inline
from importlib import reload
import utils; reload(utils)
from utils import *

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import skimage, os
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import dicom
import scipy.misc

import SimpleITK as sitk
import time
from os import listdir, walk

## Load images and get spacing for images

In [ ]:
'''
This function reads a '.mhd' file using SimpleITK and return the image array, 
origin and spacing of the image.
'''
def load_itk(filename):
    # Reads the image using SimpleITK
    itkimage = sitk.ReadImage(filename)
    
    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    ct_scan = sitk.GetArrayFromImage(itkimage)
    
    # Read the origin of the ct_scan, will be used to convert the coordinates from world to voxel and vice versa.
    origin = np.array(list(reversed(itkimage.GetOrigin())))
    
    # Read the spacing along each dimension
    spacing = np.array(list(reversed(itkimage.GetSpacing())))
    
    return ct_scan, origin, spacing

'''
This function is used to convert the world coordinates to voxel coordinates using 
the origin and spacing of the ct_scan
'''
def world_2_voxel(world_coordinates, origin, spacing):
    stretched_voxel_coordinates = world_coordinates - origin # np.absolute(world_coordinates - origin)
    voxel_coordinates = stretched_voxel_coordinates / spacing
    return voxel_coordinates

'''
This function is used to convert the voxel coordinates to world coordinates using 
the origin and spacing of the ct_scan.
'''
def voxel_2_world(voxel_coordinates, origin, spacing):
    stretched_voxel_coordinates = voxel_coordinates * spacing
    world_coordinates = stretched_voxel_coordinates + origin
    return world_coordinates

def load_itk_spacing(filename):
    # Reads the image using SimpleITK
    itkimage = sitk.ReadImage(filename)
    
      
    # Read the spacing along each dimension
    spacing = np.array(list(reversed(itkimage.GetSpacing())))
    
    return spacing

def normalizePlanes(npzarray):
    maxHU = 400.
    minHU = -1000.
    npzarray = (npzarray - minHU) / (maxHU - minHU)
    npzarray[npzarray>1] = 1.
    npzarray[npzarray<0] = 0.
    return npzarray

import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [ ]:
def seq(start, stop, step=1):
    n = int(round((stop - start)/float(step)))
    if n > 1:
        return([start + step*i for i in range(n+1)])
    else:
        return([])

'''
This function is used to create spherical regions in binary masks
at the given locations and radius.
'''
def draw_circles(image,cands,origin,spacing):
    RESIZE_SPACING = [1, 1, 1]
    image_mask = np.zeros(image.shape)
    
    for ca in cands.values:
        #get middel x-,y-, and z-worldcoordinate of the nodule
        radius = np.ceil(ca[4])/2
        coord_x = ca[1]
        coord_y = ca[2]
        coord_z = ca[3]
        image_coord = np.array((coord_z,coord_y,coord_x))

        #determine voxel coordinate given the worldcoordinate
        image_coord = world_2_voxel(image_coord,origin,spacing)
        
        #determine the range of the nodule
        noduleRange = seq(-radius, radius, RESIZE_SPACING[0])
        
        #create the mask
        for x in noduleRange:
            for y in noduleRange:
                for z in noduleRange:
                    coords = world_2_voxel(np.array((coord_z+z,coord_y+y,coord_x+x)),origin,spacing)
                    if (np.linalg.norm(image_coord-coords) * RESIZE_SPACING[0]) < radius:
                        image_mask[np.round(coords[0]),np.round(coords[1]),np.round(coords[2])] = int(1)
    return image_mask

'''
This function takes the path to a '.mhd' file as input and 
is used to create the nodule masks and segmented lungs after 
rescaling to 1mm size in all directions. It saved them in the .npz
format. It also takes the list of nodule locations in that CT Scan as 
input.
'''
def create_nodule_mask(imagePath, maskPath, cands):
    #if os.path.isfile(imagePath.replace('original',SAVE_FOLDER_image)) == False:
    img, origin, spacing = load_itk(imagePath)

    #calculate resize factor
    RESIZE_SPACING = [1, 1, 1]
    resize_factor = spacing / RESIZE_SPACING
    new_real_shape = img.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize = new_shape / img.shape
    new_spacing = spacing / real_resize
    
    #resize image
    lung_img = scipy.ndimage.interpolation.zoom(img, real_resize)
    
    # Segment the lung structure
    lung_img = lung_img + 1024
    lung_mask = segment_lung_from_ct_scan(lung_img)
    lung_img = lung_img - 1024

    #create nodule mask
    nodule_mask = draw_circles(lung_img,cands,origin,new_spacing)

    lung_img_512, lung_mask_512, nodule_mask_512 = np.zeros((lung_img.shape[0], 512, 512)), np.zeros((lung_mask.shape[0], 512, 512)), np.zeros((nodule_mask.shape[0], 512, 512))

    original_shape = lung_img.shape	
    for z in range(lung_img.shape[0]):
        offset = (512 - original_shape[1])
        upper_offset = np.round(offset/2)
        lower_offset = offset - upper_offset

        new_origin = voxel_2_world([-upper_offset,-lower_offset,0],origin,new_spacing)

        lung_img_512[z, upper_offset:-lower_offset,upper_offset:-lower_offset] = lung_img[z,:,:]
        lung_mask_512[z, upper_offset:-lower_offset,upper_offset:-lower_offset] = lung_mask[z,:,:]
        nodule_mask_512[z, upper_offset:-lower_offset,upper_offset:-lower_offset] = nodule_mask[z,:,:]

    # save images.    
    np.save(imageName + '_lung_img.npz', lung_img_512)
    np.save(imageName + '_lung_mask.npz', lung_mask_512)
    np.save(imageName + '_nodule_mask.npz', nodule_mask_512)
        

In [ ]:
data_path = "/Volumes/Backups/data/LUNA16/"

In [ ]:
patient_slice = load_itk(data_path + "subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.111172165674661221381920536987.mhd")

In [ ]:
annotations = pd.read_csv(data_path + "annotations.csv")

In [ ]:
candidates = pd.read_csv(data_path + "candidates3.csv")

In [ ]:
candidates[4:5]

In [ ]:
annotations


In [ ]:
patient_slice



In [ ]:
patient_slice[0][0][0]


In [ ]:
plt.imshow(patient_slice[0][0])


In [ ]:
plt.show()


In [ ]:
patient_slice[0][0][300]

In [ ]:
patient_slice[2]

In [ ]:
type(image)

In [ ]:
image = sitk.ReadImage(data_path + "subset0//1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260.mhd")

In [ ]:
print (image.GetSize())

In [ ]:
print (image.GetOrigin())
print (image.GetSpacing())
print (image.GetDirection())

print (image.GetNumberOfComponentsPerPixel())

In [ ]:
slice_array=slice_array.transpose(0,2,3,1)

In [ ]:
slice_array.shape

In [ ]:
sitk.GetArrayFromImage(image)[0:3].shape

In [ ]:
sitk.GetArrayFromImage(image).shape

In [ ]:
plt.imshow(sitk.GetArrayFromImage(image)[99])

In [ ]:
image.GetSpacing()

In [ ]:
image_array=sitk.GetArrayFromImage(image)

In [ ]:
(image_array).shape


In [ ]:
def get_5dslice(d_image,slice_loc):
    slice = sitk.GetArrayFromImage(d_image)[slice_loc-2:slice_loc+3]
    return slice


In [ ]:
plt.imshow(get_5dslice(image,91))

In [ ]:
new_image = get_5dslice(image,91)

In [ ]:
a = np.array([0,1,2])

In [ ]:
np.roll(a,1)

In [ ]:
def resample_image(image, image_array, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    
    real_spacing =  image.GetSpacing()
    spacing = np.array(np.roll(real_spacing,1), dtype=np.float32)

    resize_factor = spacing / new_spacing
    print(spacing)
    new_real_shape = image_array.shape * resize_factor
    new_shape = np.round(new_real_shape)
    print(new_shape)
    real_resize_factor = new_shape / image_array.shape
    new_spacing = spacing / real_resize_factor
    print(new_spacing)
    image_array2 = scipy.ndimage.interpolation.zoom(image_array, real_resize_factor, mode='nearest')
    
    return image_array2, new_spacing


In [ ]:
%time outcome = resample_image(image,new_image)

In [ ]:
for i in range(12):
    plt.imshow(outcome[0][i])
    plt.show()

In [ ]:
def get_3dslice(d_image,slice_loc):
    slice = sitk.GetArrayFromImage(d_image)[slice_loc-1:slice_loc+2]
    return slice

In [ ]:
slice = sitk.GetArrayFromImage(image)[slice_loc-1:slice_loc+2]

In [ ]:
get_3dslice(image,4).shape

In [ ]:
def create_slice_array(image):
    num_layers = image.GetSize()[2]-2
    print(num_layers)
    slice_array = np.zeros((image.GetSize()[2]-2,3,512,512))
    for i in range(num_layers):
        slice =  np.zeros((3,512,512))
        slice_array[i] = get_3dslice(image,i+1)
    slice_array = slice_array.transpose(0,2,3,1)
    return slice_array

In [ ]:
slice_array.shape


In [ ]:
%time color_array = create_slice_array(image)

In [ ]:
color_array[92].shape
plt.imshow(color_array[0])

In [ ]:
for i in range(len(slice_array)):
    plt.imshow(slice_array[i])
    plt.show()

In [ ]:
candidates.values

In [ ]:
annotations.loc[annotations['diameter_mm'].idxmax(),'coordZ']

In [ ]:
annotations

In [ ]:
annotations['diameter_mm'].max()

In [ ]:
mhd_file = (data_path + '')

In [ ]:
mhdfiles = []
spacing_list = []

for dirName, subdirList, fileList in os.walk(data_path):
    for subdirName in subdirList:
        if subdirName.startswith('sub'):
            print (subdirName)
            for dirName2,subdirList2,fileList2 in os.walk(dirName + subdirName):
                for file in fileList2:
                    if file.endswith('mhd'):
                        mhdfiles.append(dirName2 + '/' +file)
                        templist = (load_itk_spacing(dirName2 + '/' +file))
                        tempfilelist = [dirName2 + '/' + file, file]
                        for x in templist:
                            tempfilelist.append(x)
                        spacing_list.append(tempfilelist)

In [ ]:
spacing_df = pd.DataFrame(spacing_list)

In [ ]:
spacing_df.columns = ["file_loc","seriesuid",'dz','dx','dy']

In [ ]:
spacing_df['seriesuid'].replace(regex=True,inplace=True,to_replace=r'.mhd',value=r'')


In [ ]:
pd.options.display.max_colwidth = 100

In [ ]:
spacing_df.hist(column='dz')

In [ ]:
candidates


In [ ]:
cand

In [ ]:
type(candidates)

In [ ]:
# get candidates
for cand in candidates:
    worldCoord = np.asarray(candidates['coordZ']),candidates['coordY'],candidates['coordY'])
    voxelCoord = worldToVoxelCoord(worldCoord, numpyOrigin, numpySpacing)
    voxelWidth = 100
    patch = numpyImage[voxelCoord[0],voxelCoord[1]-voxelWidth/2:voxelCoord[1]+voxelWidth/2,voxelCoord[2]-voxelWidth/2:voxelCoord[2]+voxelWidth/2]
    patch = normalizePlanes(patch)
    print ('data')
    print (worldCoord)
    print (voxelCoord)
    print (patch)
    outputDir = 'patches/'
    plt.imshow(patch, cmap='gray')
    plt.show()
    Image.fromarray(patch*255).convert('L').save("/Volumes/Backups/data/LUNA16/" + outputDir + 'patch_' + str(worldCoord[0])  + '_' + str(worldCoord[1]) + '_' + str(worldCoord[2]) + '.tiff')


In [ ]:
annotations.head()

In [ ]:
candidates.head()

In [ ]:
candidates_full = pd.read_csv(data_path + "candidates.csv")

In [ ]:
candidates_tumor = candidates_full.loc[candidates_full['class'] == 1]

In [ ]:
len (candidates_tumor)

In [ ]:
candidates_tumor.head()

In [ ]:
candidates_class1_df = pd.merge(spacing_df,candidates_tumor, on='seriesuid',how='outer').dropna()

In [ ]:
candidates_class1_df[candidates_class1_df.seriesuid == '1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886']

In [ ]:
# get candidates
for   index,row in candidates_class1_df.iterrows():
    numpyImage, numpyOrigin, numpySpacing = load_itk(row[0])
    print (numpyImage.shape)
    print (numpyOrigin.shape)
    print (numpyOrigin)
    
    print (numpySpacing)
    worldCoord =([row[7],row[6],row[5]])
    voxelCoord = world_2_voxel(worldCoord, numpyOrigin, numpySpacing)
    voxelWidth = 224
    patch = numpyImage[voxelCoord[0],voxelCoord[1]-voxelWidth/2:voxelCoord[1]+voxelWidth/2,voxelCoord[2]-voxelWidth/2:voxelCoord[2]+voxelWidth/2]
    patch = normalizePlanes(patch)
    print ('data')
    print (worldCoord)
    print (voxelCoord)
    print (patch)
    print (numpyOrigin)
    outputDir = 'patches/'
    plt.imshow(patch*255, cmap='gray', vmin=0.,vmax=255.)
    plt.show()
    full_img = numpyImage[voxelCoord[0]]
    full_img = normalizePlanes(full_img)
    plt.imshow(full_img*255, cmap='gray', vmin=0.,vmax=255.)
    plt.show()

In [ ]:
candidates_class1_df

In [ ]:
candidates_class1_df[0:2]

In [ ]:
len(candidates_class1_df)

In [ ]:
len(candidates_full[candidates_full['class'] == 0])

In [ ]:
cols = candidates_class1_df.columns.tolist()


In [ ]:
cols


In [ ]:
cols = [cols[0],cols[1],cols[5],cols[6],cols[7],cols[2],cols[3],cols[4],cols[8]]